## This notebook will help you train a vanilla Point-Cloud AE with the basic architecture we used in our paper.
    (it assumes latent_3d_points is in the PYTHONPATH and the structural losses have been compiled)

In [32]:

import tflearn
import numpy as np

import os.path as osp
import tensorflow as tf
from latent_3d_points.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latent_3d_points.src.autoencoder import Configuration as Conf
from latent_3d_points.src.point_net_ae import PointNetAutoEncoder

from latent_3d_points.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from latent_3d_points.src.tf_utils import reset_tf_graph
import ipyvolume as ipv

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

Define Basic Parameters

In [18]:
top_out_dir = '../data/'                        # Use to write Neural-Net check-points etc.
top_in_dir = '../data/shape_net_core_uniform_samples_2048/' # Top-dir of where point-clouds are stored.


model_dir = osp.join(top_out_dir, 'single_class_ae_plane')
experiment_name = 'single_class_ae'
n_pc_points = 2048                              # Number of points per model.
bneck_size = 128                                # Bottleneck-AE size
ae_loss = 'chamfer'                             # Loss to optimize: 'emd' or 'chamfer'
class_name = 'airplane'

Load Point-Clouds

In [19]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)    # e.g. /home/yz6/code/latent_3d_points/data/shape_net_core_uniform_samples_2048/02691156
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

4045 pclouds were loaded. They belong in 1 shape-classes.


Load default training parameters (some of which are listed here). For more details please use print, etc.

    'batch_size': 50   
    
    'denoising': False     (# by default AE is not denoising)

    'learning_rate': 0.0005

    'z_rotate': False      (# randomly rotate models of each batch)
    
    'loss_display_step': 1 (# display loss at end of this many epochs)
    'saver_step': 10       (# how many epochs to save neural-network)

In [20]:

train_dir = create_dir(osp.join(top_out_dir, experiment_name))
out_dir = create_dir(osp.join(top_in_dir, syn_id + '_hidden'))
train_params = default_train_params()
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)

In [21]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            train_dir = train_dir,
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5              # How often to evaluate/print out loss on held_out data (if any).

In [22]:
reset_tf_graph()
print tf.get_default_graph().as_graph_def()

versions {
  producer: 24
}



Build AE Model.

In [23]:

ae = PointNetAutoEncoder(conf.experiment_name, conf)

Building Encoder
encoder_conv_layer_0 conv params =  256 bnorm params =  128
Tensor("single_class_ae_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

encoder_conv_layer_1 conv params =  8320 bnorm params =  256
Tensor("single_class_ae_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_2 conv params =  16512 bnorm params =  256
Tensor("single_class_ae_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_3 conv params =  33024 bnorm params =  512
Tensor("single_class_ae_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
output size: 524288 

encoder_conv_layer_4 conv params =  32896 bnorm params =  256
Tensor("single_class_ae_2/Relu_4:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

Tensor("single_class_ae_2/Max:0", shape=(?, 128), dtype=float32)
Building Decoder
decoder_fc_0 FC params =  33024 Tensor("single_class_ae_2/Relu_5:0", shape=(?, 256), dtype=float32)
output size: 256 

In [24]:
model_dir

'../data/single_class_ae_plane'

Train the AE (save output to train_stats.txt) 

In [27]:
ae.restore_model(model_dir, 500)

INFO:tensorflow:Restoring parameters from ../data/single_class_ae_plane/models.ckpt-500


In [96]:
code_file_path = '../data/lgan_single_class_ae_plane/hidden.npy'

In [97]:
hidden_z = np.load(code_file_path)
fake = ae.decode(hidden_z)

In [111]:
obj = fake[12]
x, y, z = obj[:, 0], obj[:, 1], obj[:, 2]
ipv.xyzlim(-0.5, 0.5)
ipv.quickscatter(x + 0.5 , y + 0.5, z +0.5, size=0.8, marker="sphere", data_max = 0.5)

VBox(children=(Figure(camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white', dtype='|S5'), geo=u'sphere', size=array(0.8), size_selected=array(2.6), x=array([0.6294457 , 0.5619705 , 0.21858114, ..., 0.5114307 , 0.22618586,
       0.4231701 ], dtype=float32), y=array([0.51284146, 0.5678689 , 0.49344897, ..., 0.41212475, 0.5506652 ,
       0.31748137], dtype=float32), z=array([0.42696244, 0.43777776, 0.52570444, ..., 0.4304502 , 0.46211037,
       0.44228742], dtype=float32))], style={'box': {'visible': True}, 'axes': {'color': 'black', 'visible': True, 'ticklabel': {'color': 'black'}, 'label': {'color': 'black'}}, 'background-color': 'white'}, tf=None, width=400, xlim=[0.0, 1.0], ylim=[0.0, 1.0], zlim=[0.0, 1.0]),))

In [29]:
hidden_code_file = 
n_examples = all_pc_data.num_examples
batch_size = conf.batch_size
n_batches = int(n_examples / batch_size)
latent_list=list()
for _ in xrange(n_batches):
    feed_pc, feed_model_names, _ = all_pc_data.next_batch(batch_size)
    latent_codes = ae.transform(feed_pc)
    latent_list.append(latent_codes)
    
latent = np.concatenate(latent_list, axis=0)
    

In [30]:
np.save(osp.join(model_dir, 'hidden.npy'), latent)

Get some reconstuctions and latent-codes.

In [ ]:


reconstructions = ae.reconstruct(feed_pc)
